In [20]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint
from copy import deepcopy
import dill

In [2]:
# ## from http://users.telenet.be/d.rijmenants/en/enigmatech.htm
# entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'  #(rotor right side)   
# I     = 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'
# II    = 'AJDKSIRUXBLHWTMCQGZNPYFVOE'
# III   = 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
# IV    = 'ESOVPZJAYQUIRHXLNFTGKDCMWB'
# V     = 'VZBRGITYUPSDNHLXAWMJQOFECK'
# ## Additional rotors used by Kriegsmarine M3 and M4 only:
# VI    = 'JPGVOUMFYQBENHZRDKASXLICTW'
# VII   = 'NZJHGRCXMYSWBOUFAIVLPEKQDT'
# VIII  = 'FKQHTLXOCBJSPDZRAMEWNIUYGV'

# ## reflectors, standard army & airforce
# contacts   = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
# breflector = 'YRUHQSLDPXNGOKMIEBFZCWVJAT'
# creflector = 'FVPJIAOYEDRZXWGCTKUQSBNMHL'

{'I':'Q','II':'E','III':'V','IV':'J','V':'Z'}

Notches
Rotor    Notch 	Window 	next left rotor steps when rotor steps from/to

I        Y      Q       Q -> R
II       M      E       E -> F
III      D      V       V -> W
IV       R      J       J -> K
V        H      Z       Z -> A
VI-VIII  H+U    Z, M    Z -> A and M -> N

Rotor 	Notch 	Window 	next left rotor steps when rotor steps from/to

I
II
III
IV
V
VI VII VIII 

	

Y
M
D
R
H
H + U

	

Q
E
V
J
Z
Z and M

	

Q -> R
E -> F
V -> W
J -> K
Z -> A
Z -> A and M -> N

In [3]:
## initial data needed to generate setup of rotors through Bombe Scrambler
# entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
# raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
# forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}
# rev_rotors = {}
# for r in raw_rotors.keys():
#     working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
#     rev_rotors[r] = [working[k] for k in sorted(working.keys())]
# reflectors = {'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 
#               'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}

entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
raw_rotors = {'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}
notches = {'I':'Q','II':'E','III':'V','IV':'J','V':'Z'}
## forward rotors is the forward in:out pairings of each rotor as the character index of the A-Z ascii alphabet stored in 'entry'
forward_rotors = {k:[ascii_uppercase.index(c) for c in raw_rotors[k]] for k in raw_rotors.keys()}

## couldn't see a straightforward list comprehension for the reverse - the in:out pairing for when the current flows
## back from the reflector to the final output. Hopefully this two-step for loop isn't too slow
rev_rotors = {}
for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

# for k,v in rev_rotors.items():
#     print(k,':',v)

## in:out pairings for reflectors are hardcoded in, were originally generated by code
reflectors = {'B': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'C': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}
# print(reflectors)

In [4]:
class ScramblerBombe:
    
    def __init__(self,left_rotor,middle_rotor,right_rotor,reflector,menu_link='ZZZ'):
        """rotors must be strings referring to either ['I','II','III','IV','V']
        reflector must be string, one of either ['B','C']"""
        
        self.right_rotor = right_rotor
        self.middle_rotor = middle_rotor
        self.left_rotor = left_rotor
        self.reflector = reflectors[reflector]
        self.menu_link = menu_link
        self.middle_notch = entry.index(notches[self.middle_rotor])   ## point if right rotor reaches will trigger middle rotor to step
        self.left_notch = entry.index(notches[self.left_rotor])  ## point if middle rotor reaches will trigger left rotor to step
        self.pos_left_rotor, self.pos_mid_rotor, self.pos_rgt_rotor = (ascii_uppercase.index(m) for m in menu_link.upper())
        self.in_status = {char:0 for char in entry}
        self.out_status = {char:0 for char in entry}
        
    def once_thru_scramble(self,start_character, direction, first_rotor, pos1, second_rotor, pos2, 
                       third_rotor, pos3):
        """ start_character must be single ASCII character A-Z
        direction is either 'forward' or 'back' """
        if direction == 'forward':
            usedict = {k:v for k,v in forward_rotors.items()}
        elif direction == 'back':
            usedict = {k:v for k,v in rev_rotors.items()}
        else:
            print('only forward or back for direction')
            return 'wtf'

        start_character = start_character.upper()
        entry_pos = entry.index(start_character)
        fst_pos_modifier = (26 + pos1 - 0)%26
        fst_in = (entry_pos + fst_pos_modifier)%26
        fst_out = usedict[first_rotor][fst_in]
        ch1o = entry[fst_out]

        scd_pos_modifier = (26 + pos2 - pos1)%26
        scd_in = (fst_out + scd_pos_modifier)%26
        ch2i = entry[scd_in]
        scd_out = usedict[second_rotor][scd_in]
        ch2o = entry[scd_out]

        thd_pos_modifier = (26 + pos3 - pos2)%26
        thd_in = (scd_out + thd_pos_modifier)%26
        ch3i = entry[thd_in]
        thd_out = usedict[third_rotor][thd_in]
        ch3o = entry[thd_out]

#         if direction == 'forward':
#             print(f"{start_character} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
#         elif direction == 'back':
#             print(f"{start_character} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")

        return ch3o
    
    def full_scramble(self,in_ch):
        in_ch = in_ch.upper()
        left_rotor = self.left_rotor
        middle_rotor = self.middle_rotor
        right_rotor = self.right_rotor
        rflector = self.reflector
        ## first run right to left through scrambler
        forward_run = self.once_thru_scramble(in_ch, direction='forward', first_rotor=right_rotor, pos1=self.pos_rgt_rotor, 
                                              second_rotor=middle_rotor, pos2=self.pos_mid_rotor, 
                                              third_rotor=left_rotor, pos3=self.pos_left_rotor)

        ## reflector back around for return
        rfi_pos_mod = (26 + 0 - self.pos_left_rotor)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
        rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
        chri = entry[rf_in]
        mirrored = rflector[chri]

#         print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")

        ## second run back left to right thru scrambler
        back_run = self.once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=self.pos_left_rotor, 
                                      second_rotor=middle_rotor, pos2=self.pos_mid_rotor, third_rotor=right_rotor, pos3=self.pos_rgt_rotor)

        bk_out = entry.index(back_run)
        bko_pos_mod = (26 + 0 - self.pos_rgt_rotor)%26  ## as above, '0' just reflects that the entry interface doesn't move
        bk_final = (bk_out + bko_pos_mod)%26
        final = entry[bk_final]
#         print('RR back out:  ', back_run, '-->', final)
#         print(in_ch,"-->",final)
        return final
    
    def rotor_step(self,rotor_position):
        """"""
        if rotor_position == 25:
            rotor_position = 0
        else:
            rotor_position += 1
        return rotor_position
    
    def step_enigma(self):
        """Just acts on itself, steps the rotors"""
        
#         print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")
        
        if self.pos_rgt_rotor == self.middle_notch and self.pos_mid_rotor == self.left_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
            self.pos_left_rotor = self.rotor_step(self.pos_left_rotor)
            print('--- left & middle rotor step')
        elif self.pos_rgt_rotor == self.middle_notch:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            self.pos_mid_rotor = self.rotor_step(self.pos_mid_rotor)
            print('--- middle rotor step ---')
        else:
            self.pos_rgt_rotor = self.rotor_step(self.pos_rgt_rotor)
            
        print(f"Lpos={self.pos_left_rotor}, Mpos={self.pos_mid_rotor}, Rpos={self.pos_rgt_rotor}")

    def update(self):
        """idea here is that the scrambler will check each of the 26 connections to see if they
        are live, and if so pass it through itself to live up the connection on the other side of 
        the scrambler"""
        
        for char, status in self.in_status.items():
            if status == 0:
                pass
            else:
                self.out_status[self.full_scramble(char)] = 1
                
        for char, status in self.out_status.items():
            if status == 0:
                pass
            else:
                self.in_status[self.full_scramble(char)] = 1
            

            
            

In [5]:
eb1 = ScramblerBombe(left_rotor='II',middle_rotor='V',right_rotor='III',reflector='C',menu_link='BCD')

In [6]:
# eb1.in_status

In [9]:
eb1.in_status['E'] = 1
eb1.in_status['H'] = 1
eb1.in_status['P'] = 1
eb1.in_status['X'] = 1
eb1.in_status

{'A': 0,
 'B': 0,
 'C': 0,
 'D': 0,
 'E': 1,
 'F': 0,
 'G': 0,
 'H': 1,
 'I': 0,
 'J': 0,
 'K': 0,
 'L': 0,
 'M': 0,
 'N': 0,
 'O': 0,
 'P': 1,
 'Q': 0,
 'R': 0,
 'S': 0,
 'T': 0,
 'U': 0,
 'V': 0,
 'W': 0,
 'X': 1,
 'Y': 0,
 'Z': 0}

In [8]:
# eb1.out_status

In [10]:
eb1.update()
eb1.out_status

{'A': 0,
 'B': 0,
 'C': 0,
 'D': 0,
 'E': 1,
 'F': 0,
 'G': 1,
 'H': 1,
 'I': 0,
 'J': 0,
 'K': 0,
 'L': 0,
 'M': 0,
 'N': 0,
 'O': 0,
 'P': 0,
 'Q': 0,
 'R': 0,
 'S': 0,
 'T': 0,
 'U': 0,
 'V': 0,
 'W': 1,
 'X': 0,
 'Y': 0,
 'Z': 0}

In [ ]:
%%time
for i in range(26**3):
    eb1.full_scramble('Q')
    eb1.step_enigma()

In [ ]:
word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [ ]:
def only_ascii(instring):
    """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
    newstring = ''
    for character in instring:
        if character in ascii_letters:
            newstring += character.upper()
    return newstring

In [ ]:
only_ascii(word)

In [21]:
with open('menu_transfer.pickle', 'rb') as infile:
    menu = dill.load(infile)
menu

{9: {'in': 'A',
  'out': 'W',
  'menu_link': 'ZZI',
  'conx_in': {6: 'out', 17: 'in', 4: 'in'},
  'conx_out': {2: 'in'}},
 2: {'in': 'W',
  'out': 'B',
  'menu_link': 'ZZB',
  'conx_in': {9: 'out'},
  'conx_out': {7: 'in'}},
 7: {'in': 'B',
  'out': 'L',
  'menu_link': 'ZZG',
  'conx_in': {2: 'out'},
  'conx_out': {14: 'in'}},
 14: {'in': 'L',
  'out': 'U',
  'menu_link': 'ZZN',
  'conx_in': {7: 'out'},
  'conx_out': {10: 'in'}},
 10: {'in': 'U',
  'out': 'O',
  'menu_link': 'ZZJ',
  'conx_in': {14: 'out'},
  'conx_out': {0: 'in'}},
 0: {'in': 'O',
  'out': 'Y',
  'menu_link': 'ZZZ',
  'conx_in': {10: 'out'},
  'conx_out': {6: 'in'}},
 6: {'in': 'Y',
  'out': 'A',
  'menu_link': 'ZZF',
  'conx_in': {0: 'out'},
  'conx_out': {9: 'in', 17: 'in', 4: 'in'}},
 17: {'in': 'A',
  'out': 'C',
  'menu_link': 'ZZQ',
  'conx_in': {9: 'in', 6: 'out', 4: 'in'},
  'conx_out': {1: 'in'}},
 1: {'in': 'C',
  'out': 'H',
  'menu_link': 'ZZA',
  'conx_in': {17: 'out'},
  'conx_out': {3: 'in'}},
 3: {'in'

In [12]:
class Bombe:
    
    def __init__(self,left_rotor='I',middle_rotor='II',right_rotor='III',reflector='B'):
        self.left_rotor = left_rotor
        self.middle_rotor = middle_rotor
        self.right_rotor = right_rotor
        self.reflector = reflector
        self.scramblers = {}
        
    def setup(self,menu):
        for key,val in menu.items():
            self.scramblers[key] = ScramblerBombe(self.left_rotor,self.middle_rotor,self.right_rotor,self.reflector,menu_link=val['menu_link'])
        

In [13]:
tb1 = Bombe('I','II','III','B')

In [14]:
tb1.setup(menu)

In [15]:
tb1.scramblers

{9: <__main__.ScramblerBombe at 0x10e1df5f8>,
 2: <__main__.ScramblerBombe at 0x10e1df5c0>,
 7: <__main__.ScramblerBombe at 0x10e1df978>,
 14: <__main__.ScramblerBombe at 0x10e1df8d0>,
 10: <__main__.ScramblerBombe at 0x10e1df0b8>,
 0: <__main__.ScramblerBombe at 0x10e1df320>,
 6: <__main__.ScramblerBombe at 0x10e1dfd30>,
 17: <__main__.ScramblerBombe at 0x10e1dfbe0>,
 1: <__main__.ScramblerBombe at 0x10e1df9e8>,
 3: <__main__.ScramblerBombe at 0x10e1dfa20>,
 4: <__main__.ScramblerBombe at 0x10e1df400>,
 5: <__main__.ScramblerBombe at 0x10e1dfef0>}

In [23]:
tb1.scramblers[9].in_status

{'A': 0,
 'B': 0,
 'C': 0,
 'D': 0,
 'E': 0,
 'F': 0,
 'G': 0,
 'H': 0,
 'I': 0,
 'J': 0,
 'K': 0,
 'L': 0,
 'M': 0,
 'N': 0,
 'O': 0,
 'P': 0,
 'Q': 0,
 'R': 0,
 'S': 0,
 'T': 0,
 'U': 0,
 'V': 0,
 'W': 0,
 'X': 0,
 'Y': 0,
 'Z': 0}